# 4 Modeling 

## 4.1 Contents<a id='4.1_Contents'></a>
* [4 Modeling](#4_Modeling)
  * [4.1 Contents](#4.1_Contents)
  * [4.2 Introduction](#4.2_Introduction)
  * [4.3 Imports](#4.3_Imports)
  * [4.4 Load The Data](#4.4_Load_The_Data)
  * [4.5 Initial Average Mode](#4.5_Average_Model)
  * [4.6 Metrics](#4.7_Metrics)
  * [4.7 Models](#4.7_Models)
      * [4.7.1 Linear Model](#4.7.1_Linear_Model)    
      * [4.7.2 Nearest Neighbor](#4.7.2_Nearest_Neighbor)    
      * [4.7.3 RandomForest](#4.7.3_RandomForest)    
      * [4.7.4 GradientBoosting](#4.7.4_Gradient)    
  * [4.8 Hyper-parameter Tuning Gradient Boost](#4.11_Hyper_parameter_Tuning)
  * [4.9 Best Model](#4.9_Best_Model)
  * [4.10 Summary](#4.10_Summary)


## 4.2 Introduction<a id='4.2_Introduction'></a>
#### In this notebook we take our preproccessed data from our Preprocessing notebook and try different models to see which yield the best performance and tune our hyper-parameters to maximize best results. Quick disclaimer the data used in the prior notebooks have been superseded by a new dataframe, while the dataset is the same I went back and redid my final dataset which has more observation and a different set of features. I felt that while the world I did originally was fine, but I needed to test few parameters and different features so I will be using a new feature set but for the most part everything else is the same. I also implement a MICE imputation method rather than just imputation of mean values for each feature set

#### With the goal to make the best model which predicts house prices in the NYC market for capital fortune we have done a lot of data work cleaning and wrangling a very messy zillow dataset with various missing values and messy feature set. But after some tedious work we were able to establish a workable dataset to train our model. 

## 4.3 Imports<a id='4.3_Imports'></a>

In [126]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy.stats as stats
import matplotlib.ticker as tick
import sklearn.model_selection

from operator import itemgetter
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier)
import xgboost as xgb

import featuretools as ft
from sklearn import neighbors, datasets, preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve

class style:
   BOLD = '\033[1m'
   END = '\033[0m'

In [72]:
## This is pulled from scikit learn source I do not own this code
from sklearn.utils.validation import check_consistent_length, check_array

def mape(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):
    """Mean absolute percentage error regression loss.
    Note here that we do not represent the output as a percentage in range
    [0, 100]. Instead, we represent it in range [0, 1/eps]. Read more in the
    :ref:`User Guide <mean_absolute_percentage_error>`.
    .. versionadded:: 0.24
    Parameters
    ----------
    y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
        Estimated target values.
    sample_weight : array-like of shape (n_samples,), default=None
        Sample weights.
    multioutput : {'raw_values', 'uniform_average'} or array-like
        Defines aggregating of multiple output values.
        Array-like value defines weights used to average errors.
        If input is list then the shape must be (n_outputs,).
        'raw_values' :
            Returns a full set of errors in case of multioutput input.
        'uniform_average' :
            Errors of all outputs are averaged with uniform weight.
    Returns
    -------
    loss : float or ndarray of floats in the range [0, 1/eps]
        If multioutput is 'raw_values', then mean absolute percentage error
        is returned for each output separately.
        If multioutput is 'uniform_average' or an ndarray of weights, then the
        weighted average of all output errors is returned.
        MAPE output is non-negative floating point. The best value is 0.0.
        But note the fact that bad predictions can lead to arbitarily large
        MAPE values, especially if some y_true values are very close to zero.
        Note that we return a large value instead of `inf` when y_true is zero.
    Examples
    --------
    >>> from sklearn.metrics import mean_absolute_percentage_error
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.3273...
    >>> y_true = [[0.5, 1], [-1, 1], [7, -6]]
    >>> y_pred = [[0, 2], [-1, 2], [8, -5]]
    >>> mean_absolute_percentage_error(y_true, y_pred)
    0.5515...
    >>> mean_absolute_percentage_error(y_true, y_pred, multioutput=[0.3, 0.7])
    0.6198...
    """
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

def _check_reg_targets(y_true, y_pred, multioutput, dtype="numeric"):
    """Check that y_true and y_pred belong to the same regression task.
    Parameters
    ----------
    y_true : array-like
    y_pred : array-like
    multioutput : array-like or string in ['raw_values', uniform_average',
        'variance_weighted'] or None
        None is accepted due to backward compatibility of r2_score().
    Returns
    -------
    type_true : one of {'continuous', continuous-multioutput'}
        The type of the true target data, as output by
        'utils.multiclass.type_of_target'.
    y_true : array-like of shape (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape (n_samples, n_outputs)
        Estimated target values.
    multioutput : array-like of shape (n_outputs) or string in ['raw_values',
        uniform_average', 'variance_weighted'] or None
        Custom output weights if ``multioutput`` is array-like or
        just the corresponding argument if ``multioutput`` is a
        correct keyword.
    dtype : str or list, default="numeric"
        the dtype argument passed to check_array.
    """
    check_consistent_length(y_true, y_pred)
    y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)

    if y_true.ndim == 1:
        y_true = y_true.reshape((-1, 1))

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape((-1, 1))

    if y_true.shape[1] != y_pred.shape[1]:
        raise ValueError("y_true and y_pred have different number of output "
                         "({0}!={1})".format(y_true.shape[1], y_pred.shape[1]))

    n_outputs = y_true.shape[1]
    allowed_multioutput_str = ('raw_values', 'uniform_average',
                               'variance_weighted')
    if isinstance(multioutput, str):
        if multioutput not in allowed_multioutput_str:
            raise ValueError("Allowed 'multioutput' string values are {}. "
                             "You provided multioutput={!r}".format(
                                 allowed_multioutput_str,
                                 multioutput))
    elif multioutput is not None:
        multioutput = check_array(multioutput, ensure_2d=False)
        if n_outputs == 1:
            raise ValueError("Custom weights are useful only in "
                             "multi-output cases.")
        elif n_outputs != len(multioutput):
            raise ValueError(("There must be equally many custom weights "
                              "(%d) as outputs (%d).") %
                             (len(multioutput), n_outputs))
    y_type = 'continuous' if n_outputs == 1 else 'continuous-multioutput'

    return y_type, y_true, y_pred, multioutput

## 4.4 Load The Data<a id='4.4_Load_The_Data'></a>

In [27]:
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv')
y_test = pd.read_csv('../data/y_test.csv')

#### All of our data is imputed for their missing values utlizing the MICE imputation and, categorical values have been encoding to numbers, they are also not ordinal. As mentioned earlier few adjustments were made with features as few of school features were included as they had a positive impact on MAE reduction. 

In [7]:
X_train.head()

,ZipCode,latitude,livingArea,longitude,priceChangeRate,propertyTaxRate,HomeType,YearBuilt,Full_Bathrooms,Half_Bathrooms,...,Annual_Tax,Tax_Assessed_Value,school_1_distance,school_1_rating,school_1_size,school_1_s/t_ratio,school_2_distance,school_2_rating,school_2_size,school_2_s/t_ratio
0,-1.154509,-1.724627,0.858494,-2.301475,-0.026462,0.447383,0.322011,1.794010,0.165641,0.086021,...,-0.068242,0.057307,3.087951,0.329303,-0.662430,-0.476408,0.529444,1.949000,0.123038,-0.146034
1,0.646870,-0.775620,-0.134432,0.003659,-0.026439,-1.811992,-0.919517,-1.181012,-0.686736,-0.030701,...,-0.071497,-0.289346,-0.272991,-1.099026,-0.023647,0.496564,0.129008,-0.303900,-0.657712,-0.146034
2,0.629343,-0.266081,0.475534,-0.201490,-0.026369,-1.811992,-0.919517,-0.874309,0.165641,-0.078314,...,-0.076955,0.724850,-0.578531,-0.622916,-0.978460,-0.476408,-0.805345,0.146680,-1.247349,-2.674455
3,-0.846814,1.863459,-0.572493,0.483297,-0.026454,1.268973,-2.161045,1.579318,-1.229475,-0.078314,...,-0.068559,0.102184,-0.578531,-0.622916,0.615136,0.496564,-0.538387,-1.655639,-0.659193,-0.146034
4,0.393703,0.550104,-1.033406,0.108092,-0.026462,0.139286,0.942775,-0.720957,-0.531917,0.086021,...,-0.077763,0.603266,0.032549,0.805413,0.961423,0.010078,-0.137950,1.047840,-0.395487,-0.146034


In [29]:
ss = StandardScaler()

X_train = pd.DataFrame(ss.fit_transform(X_train),columns=X_train.columns,index=X_train.index)
X_test = pd.DataFrame(ss.transform(X_test),columns=X_test.columns,index=X_test.index)

## 4.5 Average Model<a id='4.5_Average_Model'></a>

In [22]:
train_mean = [y_train.mean()] * len(y_train)
test_mean = [y_train.mean()] * len(y_test)

print("MAE of Y_train Average prediction's:",mean_absolute_error(y_train, train_mean))
print("MAE of Y_test Average prediction's:",mean_absolute_error(y_test, test_mean))

MAE of Y_train Average prediction's: 496524.566655386
MAE of Y_test Average prediction's: 495842.7715914151


## 4.6 Metrics<a id='4.6_Metrics'></a>

#### - Metrics used will be     
     - R²- R² shows how well terms (data points) fit a curve or line.
     - MAE -Mean absolute error
     - MSE -Mean squared error
     - RMSE- Root mean squared error. It is the square root of the MSE.
     - Mean Absolute Percentage Error (MAPE)
     
Ideally you like to see all the metrics improve with each respective metric but that is not always the case for that reason we will be using MAE as the main metric


## 4.7 Models<a id='4.7_Models'></a>

### 4.7.1 Linear Model<a id='4.7.1_Linear_Model'></a>

In [74]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)

lr_pred = lr.predict(X_test)

In [83]:
print('R2 ',r2(y_test, lr_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, lr_pred)) + style.END)
print('MSE ',mse(y_test, lr_pred))
print('RMSE ',np.sqrt(mae(y_test, lr_pred)))
print('MAPE ',(mape(y_test, lr_pred)))

R2  0.3476670695998434
MAE  379598.388210031
MSE  532111095846.1924
RMSE  616.115564005675
MAPE  0.5354992708284384


### 4.7.2 Nearest Neighbor<a id='4.7.2_Nearest_Neighbor'></a>

In [76]:
from sklearn.neighbors import KNeighborsRegressor

nn =  KNeighborsRegressor(n_neighbors=2)
nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)

R2  0.577790505087675
MAE  261900.7622478386
MSE  344398307282.5273
RMSE  511.762408005745
RMSLE  0.4104330214552605


In [84]:
print('R2 ',r2(y_test, nn_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, nn_pred)) + style.END)
print('MSE ',mse(y_test, nn_pred))
print('RMSE ',np.sqrt(mae(y_test, nn_pred)))
print('MAPE ',mape(y_test, nn_pred))

R2  0.577790505087675
MAE  261900.7622478386
MSE  344398307282.5273
RMSE  511.762408005745
MAPE  0.31505120655524266


#### Lets do some parameter tuning to find the best parameter n_neighbors 

In [62]:
gk = np.arange(1,50)
gridsearch_params = {'n_neighbors': gk}

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

k = np.random.randint(1,50,25)
estimator_KNN = KNeighborsRegressor()
parameters_KNN = {
    'n_neighbors': k

}

random_search_KNN = RandomizedSearchCV(
    estimator=estimator_KNN,
    param_distributions=parameters_KNN,
    n_iter=5,
    scoring= 'neg_mean_absolute_error',
    n_jobs= -1,
    cv= 5
)
                   

In [55]:
knn_rs = random_search_KNN.fit(X_train, y_train)

In [56]:
knn_rs.best_params_

{'n_neighbors': 18}

In [85]:
nn =  KNeighborsRegressor(n_neighbors=18)
nn.fit(X_train, y_train)
nn_pred = nn.predict(X_test)

print('R2 ',r2(y_test, nn_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, nn_pred)) + style.END)
print('MSE ',mse(y_test, nn_pred))
print('RMSE ',np.sqrt(mae(y_test, nn_pred)))
print('MAPE ',mape(y_test, nn_pred))

R2  0.6243890936885258
MAE  249954.90951542323
MSE  306387615364.7142
RMSE  499.95490748208806
MAPE  0.32169259875363854


### 4.7.3 RandomForest<a id='4.7.3_RandomForest'></a>

In [86]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=100, max_depth=10)
rf.fit(X_train,y_train)

rf_pred = rf.predict(X_test)

In [87]:
print('R2 ',r2(y_test, rf_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, rf_pred)) + style.END)
print('MSE ',mse(y_test, rf_pred))
print('RMSE ',np.sqrt(mae(y_test, rf_pred)))
print('MAPE ',mape(y_test, rf_pred))

R2  0.6750328848517655
MAE  236077.92307150294
MSE  265077232341.20724
RMSE  485.87850649262407
MAPE  0.32627741299599256


In [95]:
k = np.arange(50,150,10)
md = np.arange(5,15)

estimator_RF = RandomForestRegressor()
parameters_RF = {
    'n_estimators': k,
    'max_depth' : md
}

random_search_RF = RandomizedSearchCV(
    estimator=estimator_RF,
    param_distributions=parameters_RF,
    n_iter=5,
    scoring= 'neg_mean_absolute_error',
    n_jobs= -1,
    cv= 5
)
     

In [96]:
rf_rs = random_search_RF.fit(X_train, y_train)
rf_rs.best_params_

{'n_estimators': 80, 'max_depth': 14}

In [97]:
rf = RandomForestRegressor(n_estimators=80, max_depth=14)
rf.fit(X_train,y_train)

rf_pred = rf.predict(X_test)
print('R2 ',r2(y_test, rf_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, rf_pred)) + style.END)
print('MSE ',mse(y_test, rf_pred))
print('RMSE ',np.sqrt(mae(y_test, rf_pred)))
print('MAPE ',mape(y_test, rf_pred))

R2  0.6979017175517661
MAE  222413.3680726387
MSE  246423015971.57526
RMSE  471.60721800311615
MAPE  0.3014211033041731


### 4.7.4 Gradient Boosting<a id='4.7.4_Gradient'></a>

In [98]:
gb = xgb.XGBRegressor()

gb.fit(X_train, y_train)
gb_pred = gb.predict(X_test)
gb_train = gb.predict(X_train)

In [99]:
print('R2 ',r2(y_test, gb_pred))
print(style.BOLD + 'MAE ', str(mae(y_test, gb_pred)) + style.END)
print('MSE ',mse(y_test, gb_pred))
print('RMSE ',np.sqrt(mae(y_test, gb_pred)))
print('MAPE ',mape(y_test, gb_pred))

R2  0.6965447340625471
MAE  220371.97907164492
MSE  247529913903.35767
RMSE  469.4379395315689
MAPE  0.28959049413396326


#### From all the models it seems Gradient Boosting had the best results as for as MAE now lets go further into more hyper-tuning

## 4.8 Hyperparameter Tuning<a id='4.11_Hyper_parameter_Tuning'></a>

#### From all the models we got the best results from XGBoost's gradient boost and thus we will attempt to try to squeeze better results. We will first make a log-transformation of our target variable of 'price', this will help with outliers or the extremes of very high priced homes which don't contribute to majority of the distribution. Then we will proceed to tune the parameters of XGBoost in our case it will be
    - eta which is the learning rate of our model which is the shrinkage you do at every step you are making, increasing this allows for much faster computation where as decreasing this allows for best optimum result
    - max_depth controls how complex a model becomes, more depth more likely it is to over-fit where too little leads to simple and underfitting model
    - min_child_weight is how much weight is placed on each leaf node, the larger it is the more conservative the algorithm will be.
    - subsample is the ratio of training instances, this is the amount of observations to be randomly samples for each tree. Lower values make the algorithm more conservative and prevents overfitting but too small values might lead to under-fitting.
    - colsample_bytree this is the same as subsample except this is for the columns in other words the features 
 

In [100]:
feature_names = X_train.columns

log_y_train = np.log(y_train)
log_y_test = np.log(y_test)

dtrain = xgb.DMatrix(X_train, label=log_y_train, feature_names=feature_names)

dtest = xgb.DMatrix(X_test, label=log_y_test, feature_names=feature_names)


In [113]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}

params['eval_metric'] = "mae"
num_boost_round = 999
evallist  = [(X_test,'eval'), (X_train,'train')]


In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

In [103]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=100,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,9.098022,0.000508,9.097941,0.002037
1,6.369637,0.000308,6.369566,0.001374
2,4.459765,0.000232,4.459718,0.001449
3,3.122816,0.000170,3.122906,0.001572
4,2.186898,0.000146,2.187020,0.001763
...,...,...,...,...
158,0.145527,0.001158,0.206422,0.002392
159,0.145283,0.001136,0.206397,0.002371
160,0.145076,0.001120,0.206376,0.002402
161,0.144823,0.001132,0.206335,0.002440


In [111]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(6,16)
    for min_child_weight in range(3,7)
]

In [ ]:
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=100,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)

In [ ]:
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [116]:
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=100,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)


In [ ]:
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [118]:
params['subsample'] = best_params[0]
params['colsample_bytree'] = best_params[1]

In [119]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time 
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=100,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10)
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta


Wall time: 0 ns
CV with eta=0.3
Wall time: 0 ns
	MAE 0.20504380000000003 for 160 rounds

CV with eta=0.2
Wall time: 0 ns
	MAE 0.198117 for 359 rounds

CV with eta=0.1
Wall time: 0 ns
	MAE 0.19227979999999997 for 626 rounds

CV with eta=0.05
Wall time: 0 ns
	MAE 0.18994719999999998 for 998 rounds

CV with eta=0.01
Wall time: 0 ns
	MAE 0.1956408 for 998 rounds

CV with eta=0.005
Wall time: 0 ns
	MAE 0.22772920000000002 for 998 rounds

Best params: 0.05, MAE: 0.18994719999999998


In [ ]:
print("Best params: {}, MAE: {}".format(best_params, min_mae))

In [120]:
params['eta'] = best_params
params

{'max_depth': 9,
 'min_child_weight': 3,
 'eta': 0.05,
 'subsample': 1.0,
 'colsample_bytree': 0.8,
 'objective': 'reg:squarederror',
 'eval_metric': 'mae'}

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

In [ ]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [123]:
bmlog_pred_test = best_model.predict(dtest)
bmlog_pred_train = best_model.predict(dtrain)

print("MAE of Y_train log GB prediction with HP Tuning:",mean_absolute_error(log_y_train, bmlog_pred_train))
print("MAE of Y_test log GB prediction with HP Tuning:",mean_absolute_error(log_y_test, bmlog_pred_test))

MAE of Y_train log GB prediction with HP Tuning: 0.0962218670146364
MAE of Y_test log GB prediction with HP Tuning: 0.18909769583180996


In [124]:
bm_pred_test = np.exp(bmlog_pred_test)
bm_pred_train = np.exp(bmlog_pred_train)

# Turning our prediction back to base numbers on which we did log
# Turning our prediction back to base numbers on which we did log
print('The following Metrics are after Hyper-Parameter Tuning')
print('R2 ',r2(y_test, bm_pred_test))
print(style.BOLD + 'MAE ', str(mae(y_test, bm_pred_test)) + style.END)
print('MSE ',mse(y_test, bm_pred_test))
print('RMSE ',np.sqrt(mae(y_test, bm_pred_test)))
print('MAPE ',mape(y_test, bm_pred_test))

print("MAE of Y_train GB prediction with HP Tuning:",mean_absolute_error(y_train, gb_train))
print("MAE of Y_test GB prediction with HP Tuning:",mean_absolute_error(y_test, gb_pred))
print("MAE of Y_train GB prediction with HP Tuning:",mean_absolute_error(y_train, bm_pred_train))
print("MAE of Y_test GB prediction with HP Tuning:",mean_absolute_error(y_test, bm_pred_test))


The following Metrics are after Hyper-Parameter Tuning
R2  0.7229631108758336
MAE  191770.8591292177
MSE  225980317399.05078
RMSE  437.91649789568066
MAPE  0.216080021395455
MAE of Y_train GB prediction with HP Tuning: 155064.09292400978
MAE of Y_test GB prediction with HP Tuning: 220371.97907164492
MAE of Y_train GB prediction with HP Tuning: 89318.758322242
MAE of Y_test GB prediction with HP Tuning: 191770.8591292177


## 4.9 Best Model<a id='4.9_Best_Model'></a>

In [125]:
best_model.save_model("xgb_model.model")

#### Our best model was achieved with utilizing a gradient boosting algorithm specifically we used XGBoost and we further fined tuned our hyper-parameters for our xgboost model, where we found that the following were the best parameters which lowered our prefer metric of MAE 
    - max_depth: 9
    -min_child_weight: 3
    -eta: 0.05
    -subsample: 1.0
    -colsample_bytree: 0.8


## 4.10 Summary<a id='4.10_Summary'></a>

#### We started with our training and testing data from our previous notebook(*this is from a experimental notebook where the imputation was done via gradientboosting implementation of MICE and not from notebook 4 that notebook is Preprocessing and Modeling in REDO folder) and evaluated mutiple models to test which would give the best results. 

#### With modern algorithms such as gradient boosting we know it  would yield the best results but tried linear regression, nearest neighbor regression as well as randomforest regression while they were all outdone by gradient boosting, kNN and randomforest were not so far off. After I selected the best model being XGBoost's gradient boosting, I did further hyper-parameter tuning to squeeze a little more reduction in MAE our preffered choice of metric evaluation. This yield greater results and reduced our MAE from 220371 to 191770